# Practice Session 04: Networks from text

<font size="+2" color="blue">Additional results: account types</font> 

Author: <font color="blue">Guillem Escriba Molto</font>

E-mail: <font color="blue">guillem.escriba01@estudiant.upf.edu</font>

Date: <font color="blue">02/11/2021</font>

# 1. Create the directed mention network

In [21]:
import io
import json
import gzip
import csv
import re

In [22]:
# Leave this code as-is

# Input file
COMPRESSED_INPUT_FILENAME = "CovidLockdownCatalonia.json.gz"

# These are the output files, leave as-is
OUTPUT_ALL_EDGES_FILENAME = "CovidLockdownCatalonia.csv"
OUTPUT_FILTERED_EDGES_FILENAME = "CovidLockdownCatalonia-min-weight-filtered.csv"
OUTPUT_CO_MENTIONS_FILENAME = "CovidLockdownCatalonia-co-mentions.csv"

## 1.1. Extract mentions

In [23]:
# Leave this code as-is

def extract_mentions(text):
    return re.findall("@([a-zA-Z0-9_]{5,20})", text)

print(extract_mentions("RT @Jordi: check this post by @Xavier"))

['Jordi', 'Xavier']


## 1.2. Count mentions

In [24]:
mentions_counter = {}
with gzip.open(COMPRESSED_INPUT_FILENAME, "rt", encoding="utf-8") as input_file:
    for line in input_file: # Línea por línea en el archivo va extrayendo cada tweet y de éstos su autor y el message.
        tweet = json.loads(line)
        author = tweet["user"]["screen_name"]
        message = tweet["full_text"]
        mentions = extract_mentions(message) # Extrae las menciones del tweet "@".
        for mention in mentions: # Itera por cada mención en el tweet y comprueba si esta en el diccionario. Si está el contador aumenta.
            key = (author, mention)
            if key in mentions_counter:
                mentions_counter[key] += 1
            else:
                mentions_counter[key] = 1


In [25]:
key = ("joanmariapique","catalangov")
print(mentions_counter[key])

9


In [26]:
# Leave this code as-is

with io.open(OUTPUT_ALL_EDGES_FILENAME, "w") as output_file:
    writer = csv.writer(output_file, delimiter='\t', quotechar='"')
    writer.writerow(["Source", "Target", "Weight"])
    for key in mentions_counter:
        author = key[0]
        mention = key[1]
        weight = mentions_counter[key]
        writer.writerow([author, mention, weight])

In [27]:
with io.open(OUTPUT_FILTERED_EDGES_FILENAME, "w") as output_file:
    writer = csv.writer(output_file, delimiter='\t', quotechar='"')
    writer.writerow(["Source", "Target", "Weight"])
    for key in mentions_counter:
        author = key[0]
        mention = key[1]
        weight = mentions_counter[key]
        if weight >= 2: # Esta es la única diferencia con el código anterior pues aquí se filtran solo aquellos 
            writer.writerow([author, mention, weight]) # cuyo weight, menciones, sean iguales o superiores a 2.

# 2. Visualize the directed mention network

![Mentions graph](mentions.png)

<font size="+1" color="black"> El grafo en cuestión nos muestra las menciones direccionadas (y el número de éstas, weight) entre dos o más usuarios mencionados en un tweet, en concreto solo aquellas interacciones cuyo número de veces realizadas sea igual o superior a dos, es decir, que las menciones únicas no se muestran pues es el grafo filtrado. Como se puede ver un gran número de menciones son bilaterales, sin que intervengan más de 2 usuarios.
   En este grafo podemos ver un CC central cuyos núcleos son en esencia instituciones gubernamentales o sanitarias, doctores en el ámbito y/o medios de comunicación. Dicho CC Central, tiene 699 nodos y 791 links representando un porcentaje de nodos respecto al grafo de Nc/N = 699/1600 = 43.69%, prácticamente la mitad. El siguiente CC más grande se encuentra arriba a la derecha del central, con 16 nodos y 15 links y un porcentaje de nodos respecto al grafo del 1%.
   

![Mentions graph largest connected component](mentions-largest-cc.png)

![Mentions in-degree distribution](mentions-in-degree-distribution.png)![Mentions out-degree distribution](mentions-out-degree-distribution.png)

<font size="+1" color="black">
    Este grafo al ser un componente conexo del grafo anterior resulta en, valga la redundancia, un grafo conectado(un solo componente conexo, Nc/N = 100%), es decir, que desde cualquier nodo x, se puede acceder a cualquier nodo y. Los nodos con mayor centralidad son aquellos pertenecientes a instituciones gubernamentales y sanitarias, medios de comunicación y expertos, algo previsible dada la situación de aquel momento. Podemos ver segun los dos gráficos, que el grado de salida es inferior al grado de entrada, algo bastante coerente si tenemos en cuenta que en un mismo tweet se pueden mencionar a varias personas pero que solo hay un autor. Una cosa curiosa del grafo es que tiene un clustering coefficient medio muy bajo aunque tiene cierto sentido dado que por ejemplo, una institución sanitaria puede ser muy mencionada, pero eso no significa que quienes la mencionan guarden relación alguna entre sí.</font>

# 3. Create the undirected co-mention network

In [28]:
co_mentions_counter = {}
with gzip.open(COMPRESSED_INPUT_FILENAME, "rt", encoding="utf-8") as input_file:
    for line in input_file:
        tweet = json.loads(line)
        author = tweet["user"]["screen_name"]
        message = tweet["full_text"]
        #print("%s: '%s'" % (author, message))
        mentions = extract_mentions(message)
        for mention1 in mentions:     # El código es igual al anterior solo que esta vez hay dos bucles para contar las menciones 
            for mention2 in mentions: # en ambas direcciones como una sola, el if mention 1 < mention 2, evita duplicados
                if mention1 < mention2:
                    key = (mention1, mention2)
                    if key in co_mentions_counter:
                        co_mentions_counter[key] += 1
                    else:
                        co_mentions_counter[key] = 1

In [29]:
key = ("agriculturacat","uniopagesos")
print(co_mentions_counter[key])

8


In [30]:
with io.open(OUTPUT_CO_MENTIONS_FILENAME, "w") as output_file:
    writer = csv.writer(output_file, delimiter='\t', quotechar='"')
    writer.writerow(["Source", "Target", "Weight"])
    for key in co_mentions_counter:
        author = key[0]
        mention = key[1]
        weight = co_mentions_counter[key]
        writer.writerow([author, mention, weight])


# 4. Visualize the undirected co-mention network in Cytoscape

![Co-mentions graph](co-mentions.png)

<font size="+1" color="black">
    A diferencia de los anteriores, en este grafo la dirección no es relevante, es lo mismo que x mencione a y que y mencione a x, además las menciones no están filtradas por el número de las mismas, sino que también hay menciones únicas. Como resultado de esto se muestran muchos nodos cuyo weigth es 1 puesto que por ejemplo, se puede mencionar a alguien desconocido y que este no te mencione ni tú le vuelvas a mencionar lo que tiene como resultado todos los pequeños CCs de abajo. Esto es lo que causa la gran cantidad de CCs que hay, 888. Aparte de ser un grafo desconectado con 4008 nodos y 7816 edges, posee un gran componente central 1514 nodos. Tanto el clustering coefficient como el número de vecinos, es superior al del anterior, ya que ahora las menciones unidireccionales coinciden en ambos sentido.</font>


# 5. Additional results: account types

In [31]:
import operator # Simplifica en una sola línea la ordenación del diccionario.

In [32]:
top_mentions_counter = {}
with gzip.open(COMPRESSED_INPUT_FILENAME, "rt", encoding="utf-8") as input_file:
    for line in input_file:
        tweet = json.loads(line)
        author = tweet["user"]["screen_name"]
        message = tweet["full_text"]
        mentions = extract_mentions(message)
        for mention in mentions:
            key = (mention) # A diferencia de las anteriores solo cuenta el número de veces que ha sido mencionada una cuenta.
            if key in top_mentions_counter:
                top_mentions_counter[key] += 1
            else:
                top_mentions_counter[key] = 1

# Simplifica en una sola línea la ordenación del diccionario.
top_mentions_counter = sorted(top_mentions_counter.items(), key=operator.itemgetter(1), reverse=True)





In [33]:
OUTPUT_ACCOUNT_TYPE_FILENAME = "account-type.csv"


with io.open(OUTPUT_ACCOUNT_TYPE_FILENAME, "w") as output_file:
    writer = csv.writer(output_file, delimiter='\t', quotechar='"')
    writer.writerow(["Source", "Weight"])
    i = 0
    for key in top_mentions_counter:
        i += 1
        mention = key[0]
        weight = key[1]
        writer.writerow([mention, weight])
        if i == 50:
            break;

![Account-types graph](account-type.png)


<font size="+2" color="#003300">I hereby declare that, except for the code provided by the course instructors, all of my code, report, and figures were produced by myself.</font>
